# Graded Lab: Agentic Workflows

In this lab, you will build an agentic system that generates a short research report through planning, external tool usage, and feedback integration. Your workflow will involve:

### Agents

* **Planning Agent / Writer**: Creates an outline and coordinates tasks.
* **Research Agent**: Gathers external information using tools like Arxiv, Tavily, and Wikipedia.
* **Editor Agent**: Reflects on the report and provides suggestions for improvement.

---
<a name='submission'></a>

<h4 style="color:green; font-weight:bold;">TIPS FOR SUCCESSFUL GRADING OF YOUR ASSIGNMENT:</h4>

* All cells are frozen except for the ones where you need to write your solution code or when explicitly mentioned you can interact with it.

* In each exercise cell, look for comments `### START CODE HERE ###` and `### END CODE HERE ###`. These show you where to write the solution code. **Do not add or change any code that is outside these comments**.

* You can add new cells to experiment but these will be omitted by the grader, so don't rely on newly created cells to host your solution code, use the provided places for this.

* Avoid using global variables unless you absolutely have to. The grader tests your code in an isolated environment without running all cells from the top. As a result, global variables may be unavailable when scoring your submission. Global variables that are meant to be used will be defined in UPPERCASE.

* To submit your notebook for grading, first save it by clicking the 💾 icon on the top left of the page and then click on the <span style="background-color: red; color: white; padding: 3px 5px; font-size: 16px; border-radius: 5px;">Submit assignment</span> button on the top right of the page.
---


### Research Tools

By importing `research_tools`, you gain access to several search utilities:

- `research_tools.arxiv_search_tool(query)` → search academic papers from **arXiv**  

  *Example:* `research_tools.arxiv_search_tool("neural networks for climate modeling")`

- `research_tools.tavily_search_tool(query)` → perform web searches with the **Tavily API**  

  *Example:* `research_tools.tavily_search_tool("latest trends in sunglasses fashion")`

- `research_tools.wikipedia_search_tool(query)` → retrieve summaries from **Wikipedia**  

  *Example:* `research_tools.wikipedia_search_tool("Ensemble Kalman Filter")`

Run the cell below to make them available.

In [2]:
# =========================
# Imports
# =========================

# --- Standard library 
from datetime import datetime
import re
import json
import ast


# --- Third-party ---
from IPython.display import Markdown, display
from aisuite import Client

# --- Local / project ---
import research_tools

In [3]:
import unittests

### Initialize client

Create a shared client instance for upcoming calls.

In [4]:
CLIENT = Client()

## Exercise 1: planner_agent

### Objective
Correctly set up a call to a language model (LLM) to generate a research plan.

### Instructions

1. **Focus Areas**:
   - Ensure `CLIENT.chat.completions.create` is correctly configured.
   - Pass the `model` and `messages` parameters correctly:
     - **Model**: Use `"openai:o4-mini"` by default.
     - **Messages**: Set with `{"role": "user", "content": user_prompt}`.
     - **Temperature**: Fixed at 1 for creative outputs.

### Notes

- The prompt is pre-defined and guides the LLM on task requirements.
- Only return a formatted list of steps — no extra text.

Focus on the LLM call setup to complete the task.

In [5]:
# GRADED FUNCTION: planner_agent

def planner_agent(topic: str, model: str = "openai:o4-mini") -> list[str]:
    """
    Generates a plan as a Python list of steps (strings) for a research workflow.

    Args:
        topic (str): Research topic to investigate.
        model (str): Language model to use.

    Returns:
        List[str]: A list of executable step strings.
    """

    
    # Build the user prompt
    user_prompt = f"""
    You are a planning agent responsible for organizing a research workflow with multiple intelligent agents.

    🧠 Available agents:
    - A research agent who can search the web, Wikipedia, and arXiv.
    - A writer agent who can draft research summaries.
    - An editor agent who can reflect and revise the drafts.

    🎯 Your job is to write a clear, step-by-step research plan **as a valid Python list**, where each step is a string.
    Each step should be atomic, executable, and must rely only on the capabilities of the above agents.

    🚫 DO NOT include irrelevant tasks like "create CSV", "set up a repo", "install packages", etc.
    ✅ DO include real research-related tasks (e.g., search, summarize, draft, revise).
    ✅ DO assume tool use is available.
    ✅ DO NOT include explanation text — return ONLY the Python list.
    ✅ The final step should be to generate a Markdown document containing the complete research report.

    Topic: "{topic}"
    """

    # Add the user prompt to the messages list
    messages = [{"role": "user", "content": user_prompt}]

    ### START CODE HERE ###

    # Call the LLM
    response = CLIENT.chat.completions.create( 
        # Pass in the model
        model=model,
        # Define the messages. Remember this is meant to be a user prompt!
        messages=messages,
        # Keep responses creative
        temperature=1, 
    )

    ### END CODE HERE ###

    # Extract message from response
    steps_str = response.choices[0].message.content.strip()

    # Parse steps
    steps = ast.literal_eval(steps_str)

    return steps

In [6]:
# Test your code!
unittests.test_planner_agent(planner_agent)

 All tests passed!


## Exercise 2: research_agent

### Objective
Set up a call to a language model (LLM) to perform a research task using various tools.

### Instructions

**Focus Areas**:

- **Creating a Custom Prompt**:
  - **Define the Role**: Clearly specify the role, such as "research assistant."
  - **List Available Tools** (as strings inside the prompt, not the actual functions):
    - Use `arxiv_tool` to find academic papers.
    - Use `tavily_tool` for general web searches.
    - Use `wikipedia_tool` for accessing encyclopedic knowledge.
  - **Specify the Task**: Include a placeholder in your prompt for defining the specific task that needs to be accomplished.
  - **Include Date Information**: Add a placeholder for the current date or time to provide context.

- **Creating Messages Dict**:
  - Ensure the `messages` are correctly set with `{"role": "user", "content": prompt}`.

- **Creating Tools List**:
  - Create a list of tools for use, such as `research_tools.arxiv_search_tool`, `research_tools.tavily_search_tool`, and `research_tools.wikipedia_search_tool`.

- **Correctly Setting the Call to the LLM**:
  - Pass the `model`, `messages`, and `tools` parameters accurately.
  - Set `tool_choice` to `"auto"` for automatic tool selection.
  - Limit interactions with `max_turns=6`.

### Notes

- The function provides pre-coded blocks where you need to replace placeholder values.
- The approach allows the LLM to use tools dynamically based on the task.

Focus on accurately setting the messages, tools, and LLM call parameters to complete the task.

In [7]:
# GRADED FUNCTION: research_agent

def research_agent(task: str, model: str = "openai:gpt-4o", return_messages: bool = False):
    """
    Executes a research task using tools via aisuite (no manual loop).
    Returns either the assistant text, or (text, messages) if return_messages=True.
    """
    print("==================================")  
    print("🔍 Research Agent")                 
    print("==================================")

    current_time = datetime.now().strftime('%Y-%m-%d')
    
    ### START CODE HERE ###

    # Create a customizable prompt by defining the role (e.g., "research assistant"),
    # listing tools (arxiv_tool, tavily_tool, wikipedia_tool) for various searches,
    # specifying the task with a placeholder, and including a current_time placeholder.
    prompt = f"""
    You are a Research Assistant. You have access to various tools as given below:
    Tools: 
    1) tavily_search_tool: Performs a general-purpose web search using the Tavily API.
    2) arxiv_search_tool: Searches for research papers on arXiv by query string.
    3) wikipedia_search_tool: Searches for a Wikipedia article summary by query string.
    
    You need to use above tools for search for given task below:
    Task: {task}
    Current Time: {current_time}"""
    
    # Create the messages dict to pass to the LLM. Remember this is a user prompt!
    messages = [{"role": "user", "content": prompt}]

    # Save all of your available tools in the tools list. These can be found in the research_tools module.
    # You can identify each tool in your list like this: 
    # research_tools.<name_of_tool>, where <name_of_tool> is replaced with the function name of the tool.
    tools = [research_tools.tavily_search_tool,research_tools.arxiv_search_tool,research_tools.wikipedia_search_tool]
    
    # Call the model with tools enabled
    response = CLIENT.chat.completions.create(  
        # Set the model
        model=model,
        # Pass in the messages. You already defined this!
        messages=messages,
        # Pass in the tools list. You already defined this!
        tools=tools,
        # Set the LLM to automatically choose the tools
        tool_choice="auto",
        # Set the max turns to 6
        max_turns=6
    )  
    
    ### END CODE HERE ###

    content = response.choices[0].message.content
    print("✅ Output:\n", content)

    
    return (content, messages) if return_messages else content  

In [8]:
# Test your code!
unittests.test_research_agent(research_agent)

🔍 Research Agent
✅ Output:
 Please provide me with the specific topic or query you would like me to search for, so I can gather and summarize the key references for you.
🔍 Research Agent
✅ Output:
 To find and summarize two seminal papers, let's start by searching for papers on arXiv related to influential topics in the fields of study you're interested in. We'll then summarize the results. Please specify which fields or topics you're interested in, or should I select two major fields such as artificial intelligence and quantum computing?
 All tests passed!


## Exercise 3: writer_agent

### Objective
Set up a call to a language model (LLM) for executing writing tasks like drafting, expanding, or summarizing text.

### Instructions

1. **Focus Areas**:
   - **System Prompt**:
     - Define `system_prompt` to assign the LLM the role of a writing agent focused on generating academic or technical content.
   - **System and User Messages**:
     - Create `system_msg` using `{"role": "system", "content": system_prompt}`.
     - Create `user_msg` using `{"role": "user", "content": task}`.
   - **Messages List**:
     - Combine `system_msg` and `user_msg` into a `messages` list.

### Notes

- The function is designed to produce well-structured text by setting the correct prompts.
- Temperature is set to 1.0 to allow for creative variance in the writing outputs.

Ensure the system prompt and messages are defined properly to achieve a structured output from the LLM.

In [9]:
# GRADED FUNCTION: writer_agent
def writer_agent(task: str, model: str = "openai:gpt-4o") -> str: # @REPLACE def writer_agent(task: str, model: str = None) -> str:
    """
    Executes writing tasks, such as drafting, expanding, or summarizing text.
    """
    print("==================================")
    print("✍️ Writer Agent")
    print("==================================")

    ### START CODE HERE ###
    
    # Create the system prompt.
    # This should assign the LLM the role of a writing agent specialized in generating well-structured academic or technical content
    system_prompt = f"""
    You are a Writing agents specialized in generating well-structured academic or technical content
    """

    # Define the system msg by using the system_prompt and assigning the role of system
    system_msg = {"role": "system", "content": system_prompt}

    # Define the user msg. In this case the user prompt should be the task passed to the function
    user_msg = {"role": "user", "content": task}

    # Add both system and user messages to the messages list
    messages = [system_msg, user_msg]
    
    ### END CODE HERE ###

    response = CLIENT.chat.completions.create(
        model=model, 
        messages=messages,
        temperature=1.0
    )

    return response.choices[0].message.content

In [10]:
# Test your code!
unittests.test_writer_agent(writer_agent)

✍️ Writer Agent
 All tests passed!


## Exercise 4: editor_agent

### Objective
Configure a call to a language model (LLM) to perform editorial tasks such as reflecting, critiquing, or revising drafts.

### Instructions

1. **Focus Areas**:
   - **System Prompt**:
     - Define `system_prompt` to assign the LLM the role of an editor agent whose task is to reflect on, critique, or improve drafts.
   - **System and User Messages**:
     - Create `system_msg` using `{"role": "system", "content": system_prompt}`.
     - Create `user_msg` using `{"role": "user", "content": task}`.
   - **Messages List**:
     - Combine `system_msg` and `user_msg` into a `messages` list.

### Notes

- The editor agent is tailored for enhancing the quality of text by setting an appropriate role and task in the prompts.
- Temperature is set to 0.7, balancing creativity and coherence in editorial outputs.

Ensure the system prompt and messages are accurately set up to perform effective editorial tasks with the LLM.

In [11]:
# GRADED FUNCTION: editor_agent
def editor_agent(task: str, model: str = "openai:gpt-4o") -> str:
    """
    Executes editorial tasks such as reflection, critique, or revision.
    """
    print("==================================")
    print("🧠 Editor Agent")
    print("==================================")
    
    ### START CODE HERE ###

    # Create the system prompt.
    # This should assign the LLM the role of an editor agent specialized in reflecting on, critiquing, or improving existing drafts.
    system_prompt = f"""
    You are an Editor whose task is to reflect on, critique, or improve drafts.
    """
    
    # Define the system msg by using the system_prompt and assigning the role of system
    system_msg = {"role": "system", "content": system_prompt}
    
    # Define the user msg. In this case the user prompt should be the task passed to the function
    user_msg = {"role": "user", "content": task}
    
    # Add both system and user messages to the messages list
    messages = [system_msg, user_msg]
    
    ### END CODE HERE ###
    
    response = CLIENT.chat.completions.create(
        model=model, 
        messages=messages,
        temperature=0.7 
    )
    
    return response.choices[0].message.content

In [12]:
# Test your code!
unittests.test_editor_agent(editor_agent)

🧠 Editor Agent
 All tests passed!


### 🎯 The Executor Agent

The `executor_agent` manages the workflow by executing each step of a given plan. It:

1. Decides **which agent** (`research_agent`, `writer_agent`, or `editor_agent`) should handle the step.
2. Builds context from the outputs of previous steps.
3. Sends the enriched task to the selected agent.
4. Collects and stores the results in a shared history.

👉 **Do not implement or modify this function.** It is already provided as the orchestration component of the multi-agent pipeline.

Notice that `planner_agent` might return a long list of steps. Because of this, the maximum number of steps is set to a maximum of 4 to keep running time reasonable.

In [13]:
agent_registry = {
    "research_agent": research_agent,
    "editor_agent": editor_agent,
    "writer_agent": writer_agent,
}

def clean_json_block(raw: str) -> str:
    """
    Clean the contents of a JSON block that may come wrapped with Markdown backticks.
    """
    raw = raw.strip()
    if raw.startswith("```"):
        raw = re.sub(r"^```(?:json)?\n?", "", raw)
        raw = re.sub(r"\n?```$", "", raw)
    return raw.strip()

In [14]:
def executor_agent(topic, model: str = "openai:gpt-4o", limit_steps: bool = True):

    plan_steps = planner_agent(topic)
    max_steps = 4

    if limit_steps:
        plan_steps = plan_steps[:min(len(plan_steps), max_steps)]
    
    history = []

    print("==================================")
    print("🎯 Editor Agent")
    print("==================================")

    for i, step in enumerate(plan_steps):

        agent_decision_prompt = f"""
        You are an execution manager for a multi-agent research team.

        Given the following instruction, identify which agent should perform it and extract the clean task.

        Return only a valid JSON object with two keys:
        - "agent": one of ["research_agent", "editor_agent", "writer_agent"]
        - "task": a string with the instruction that the agent should follow

        Only respond with a valid JSON object. Do not include explanations or markdown formatting.

        Instruction: "{step}"
        """
        response = CLIENT.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": agent_decision_prompt}],
            temperature=0,
        )

        raw_content = response.choices[0].message.content
        cleaned_json = clean_json_block(raw_content)
        agent_info = json.loads(cleaned_json)

        agent_name = agent_info["agent"]
        task = agent_info["task"]

        context = "\n".join([
            f"Step {j+1} executed by {a}:\n{r}" 
            for j, (s, a, r) in enumerate(history)
        ])
        enriched_task = f"""
        You are {agent_name}.

        Here is the context of what has been done so far:
        {context}

        Your next task is:
        {task}
        """

        print(f"\n🛠️ Executing with agent: `{agent_name}` on task: {task}")

        if agent_name in agent_registry:
            output = agent_registry[agent_name](enriched_task)
            history.append((step, agent_name, output))
        else:
            output = f"⚠️ Unknown agent: {agent_name}"
            history.append((step, agent_name, output))

        print(f"✅ Output:\n{output}")

    return history

In [15]:
# If you want to see the full workflow without limiting the number of steps. Set limit_steps to False
# Keep in mind this could take more than 10 minutes to complete
executor_history = executor_agent("The ensemble Kalman filter for time series forecasting", limit_steps=True)

md = executor_history[-1][-1].strip("`")  
display(Markdown(md))

🎯 Editor Agent

🛠️ Executing with agent: `research_agent` on task: Search web, Wikipedia, and arXiv for foundational and recent papers on the ensemble Kalman filter for time series forecasting
🔍 Research Agent
✅ Output:
 Here's a summary of the foundational and recent papers on the ensemble Kalman filter for time series forecasting based on searches performed via web search, arXiv, and Wikipedia:

### Web Search Results:

1. **Ensemble Kalman Filter Overview:**
   - The Ensemble Kalman Filter (EnKF) originated as a version of the Kalman filter for large problems, particularly important in ensemble forecasting as a data assimilation component. It replaces the covariance matrix with sample covariance and is applied in Bayesian update problems. [Read more](https://en.wikipedia.org/wiki/Ensemble_Kalman_filter)

2. **Understanding the Ensemble Kalman Filter:**
   - This explains computational techniques for approximating inference in state-space models using the EnKF. It's useful in Bayesia

✅ Output:
 Here are the abstracts, key findings, and citation data for the top ten most relevant papers on the ensemble Kalman filter for time series forecasting:

1. **Title:** [Ensemble Kalman Filtering Meets Gaussian Process SSM for Non-Mean-Field and Online Inference](http://arxiv.org/abs/2312.05910v5)
   - **Authors:** Zhidi Lin, Yiyong Sun, Feng Yin, Alexandre Hoang Thiéry
   - **Published:** 2023-12-10
   - **Abstract:** This paper integrates the ensemble Kalman filter (EnKF) into Gaussian Process State-Space Models (GPSSM) for improved inference accuracy and applicability for online learning. It showcases a new method eliminating extensive parameterization, offering a closed-form approximation for the evidence lower bound (ELBO). The EnKF-aided online algorithm maintains data-fit accuracy and mitigates overfitting.
   - **Key Findings:** Demonstrates superior learning and inference performance compared to existing state-of-the-art methods in diverse datasets.
   - **PDF:** [Lin


🛠️ Executing with agent: `research_agent` on task: Extract and summarize the mathematical formulation and algorithmic steps of the ensemble Kalman filter from retrieved sources
🔍 Research Agent
✅ Output:
 Here's a summary of the mathematical formulation and algorithmic steps of the ensemble Kalman filter (EnKF) based on the search results:

### Wikipedia Overview:
The **ensemble Kalman filter (EnKF)** is a recursive filter used in systems with many variables such as geophysical models. EnKF emerged from the Kalman filter for handling large-scale problems by using the sample covariance instead of the full covariance matrix. It functions under the assumption that all involved probability distributions are Gaussian, which facilitates its use in Bayesian update problems, combining prior state probabilities and new observations to obtain posterior probabilities. EnKF employs a Monte Carlo method using an ensemble of state vectors to approximate the probability density function (PDF) rather

Here are summaries of empirical case studies applying the ensemble Kalman filter (EnKF) to time series forecasting from various sources:

### Web Search Results via Tavily:
1. **Flood Forecasting:**
   - EnKF was applied to flood forecasting in the Chaohe River Basin, using satellite-based soil moisture data and runoff data. The study found that using EnKF improved prediction accuracy about flood conditions when compared to methods that did not use data assimilation. Results highlighted that assimilating both runoff and soil moisture data provided the best predictive performance. [Read More](https://www.mdpi.com/2073-4441/14/10/1555)

2. **Electricity Prices Forecasting:**
   - The EnKF was employed for forecasting electricity prices, analyzing data over a year to predict day-ahead auction returns. The model's evaluation used metrics like MAPE and RMSE compared with confidence intervals, showcasing its effectiveness in price forecasting. [Read More](https://www.scienpress.com/Upload/JSEM/Vol%209_1_2.pdf)

3. **Electricity Load Forecasting:**
   - The method was enhanced through shrinkage and regression techniques for forecasting electricity load, with applications also in forecasting ozone concentrations and snow assimilation. [Read More](https://www.sciencedirect.com/science/article/abs/pii/S0360544216303140)

4. **Geodynamo Models:**
   - EnKF was used for analyzing time-dependent geodynamo models, illustrating its application in three-dimensional, convection-driven models and indicating the broader range of physical systems it can be applied to. [Read More](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/ggge.20252)

### ArXiv Academic Search Results:
1. **LLM-Mixer for Time Series:**
   - Although not strictly an EnKF application, this study is significant as it explores large language models in time-series analysis, showing potential for future integration with EnKF techniques for hybrid forecasting models. [Read More](https://arxiv.org/pdf/2410.11674v2)

2. **Gaussian Process SSM Integration:**
   - This paper discusses integrating EnKF with Gaussian Process State-Space Models for enhancing forecasting accuracy while mitigating overfitting. [Read More](https://arxiv.org/pdf/2312.05910v5)

### Wikipedia Insights:
- While Wikipedia did not directly provide case studies on EnKF, it generally describes the Kalman filter’s broad application in control and statistical fields, particularly suitable for dynamic systems like vehicle navigation and econometric models. EnKF extends these applications into ensemble forecasting suitable for large-scale systems. [Read More](https://en.wikipedia.org/wiki/Kalman_filter)

These case studies and resources provide substantial insights into the versatility and efficacy of the ensemble Kalman filter for various time series forecasting applications.

## Check grading feedback

If you have collapsed the right panel to have more screen space for your code, as shown below:

<img src="./images/collapsed.png" alt="Collapsed Image" width="800" height="400"/>

You can click on the left-facing arrow button (highlighted in red) to view feedback for your submission after submitting it for grading. Once expanded, it should display like this:

<img src="./images/expanded.png" alt="Expanded Image" width="800" height="400"/>